In [1]:
import os
import sys

sys.path.insert(0, os.path.abspath('..'))

import cv2
import numpy as np
import pandas as pd
import networkx as nx
import ogr
import time
from scipy.spatial import Voronoi, voronoi_plot_2d

from autocnet.examples import get_path
from autocnet.graph.network import CandidateGraph

# Necessary for the cooked test data
# from unittest.mock import Mock
# from unittest.mock import MagicMock
# from autocnet.graph import edge
# from autocnet.graph import node
from plio.io import io_gdal

from autocnet.transformation.transformations import Homography
from autocnet.utils.utils import array_to_poly

from IPython.display import display

%pylab inline
figsize(20, 20)

Populating the interactive namespace from numpy and matplotlib


In [2]:
def scale_poly(point, centroid, scalar):
    point = np.asarray(point)
    centroid = centroid[:2]
    vector = ((point - centroid)*scalar) + centroid
    return (vector)

In [3]:
def reproj_point(H, point):
    """
    Reproject a pixel in one image into another image
    
    Parameters
    ----------
    H : object
        (3,3) ndarray or Homography object
        
    corner : iterable
             A 2 element iterable in the form x, y
    """
    if len(point) == 2:
        coords = np.array([point[0],point[1],1])
    elif len(point) == 3:
        coords = np.asarray(point)
        coords *= coords[-1]  # Homogenize
    
    return H.dot(coords)

In [4]:
def plot_vor(graph, source, poly_array, intersection_points, voronoi_pd):
    # For the source image
    fig1 = plt.figure()
    ax1 = fig1.add_subplot(111, aspect='equal')
    ax1.set_xlim(-150, 650)
    ax1.set_ylim(-150, 650)
    source_num = source.node_id
    source_corners = source.geodata.xy_corners
    ax1.add_patch(Polygon(source_corners, facecolor='blue',alpha=0.25, edgecolor='k'))
    
    # For every subsequent image in the intersection
    ax1.add_patch(Polygon(intersection_points, facecolor='red', alpha=0.25, edgecolor='k'))
#     if (edge.source.geodata.coordinate_transformation.this == None):
#         ax1.add_patch(Polygon(proj_corners, facecolor='red', alpha=0.25, edgecolor='k'))
#     else:
#         ax1.add_patch(Polygon(destination_corners, facecolor='red', alpha=0.25, edgecolor='k'))

#     # Draw the polygons from the voronoi as well as the points in the voronoi
    matplotlib.pyplot.scatter(voronoi_pd['x'], voronoi_pd['y'], color = 'black')
    polygons = poly_array
    for i in polygons:
        geom = i.GetGeometryRef(0)
        if geom == None:
            continue
        else:
            points = geom.GetPoints()
            plt.fill(*zip(*points), alpha = .3)

In [5]:
def compute_intersection(graph, clean_keys=[]):
    source_num = graph.edges()[0][0]
    source = graph.node[source_num]
    source_corners = source.geodata.xy_corners
    total_intersect_poly = array_to_poly(source_corners)
    orig_poly = array_to_poly(source_corners)
    
    # Method for displaying the images transformed using the homography, should be used in the visualiation
    fig1 = plt.figure()
    ax1 = fig1.add_subplot(111, aspect='equal')
    ax1.add_patch(Polygon(source_corners, facecolor='blue',alpha=0.25, edgecolor='k'))
    # End visualization section
    
    # Begin iterating through the nodes in the graph excluding the zero node
    for n in graph.nodes_iter():
        if n == source_num:
            continue
        
        # Define the edge, matches, and destination based on the zero node and the nth node
        edge = graph.edge[source_num][n]
        matches, _ = edge.clean(clean_keys=clean_keys)
        destination = edge.destination
        destination_corners = destination.geodata.xy_corners
        
        kp1 = edge.source.get_keypoint_coordinates(index=matches['source_idx'], homogeneous=True)
        kp2 = edge.destination.get_keypoint_coordinates(index=matches['destination_idx'], homogeneous=True)
        
        # If the source image has coordinate transformation data, us the footprint and
        # coordinate transforms as it will produce a more accurate intersection
        if edge.source.geodata.coordinate_transformation.this != None:
            source_footprint_poly = edge.source.geodata.footprint
            destination_footprint_poly = edge.destination.geodata.footprint

            intersection_poly = destination_footprint_poly.Intersection(source_footprint_poly)

        # Else, use the homography transform to get an intersection of the two images
        else:
            H, mask = cv2.findHomography(kp2.values,
                                         kp1.values,
                                         cv2.RANSAC,
                                         2.0)

            proj_corners = []
            for c in destination_corners:
                x, y, h = reproj_point(H, c)
                x /= h
                y /= h
                h /= h
                proj_corners.append((x, y))

            proj_poly = array_to_poly(proj_corners)

            intersection_poly = orig_poly.Intersection(proj_poly)
            
        # Intersect the newly calculated intersection with the current total intersection
        # to get the new bound of the overlap area
        total_intersect_poly = intersection_poly.Intersection(total_intersect_poly)
            
        # Method for displaying the images transformed using the homography, should be used in the visualiation
        if (edge.source.geodata.coordinate_transformation.this == None):
            ax1.add_patch(Polygon(proj_corners, facecolor='red', alpha=0.25, edgecolor='k'))
        else:
            ax1.add_patch(Polygon(destination_corners, facecolor='red', alpha=0.25, edgecolor='k'))        
        # End visualization section
        
    return total_intersect_poly

In [6]:
def vor(graph, clean_keys=[], s = 30, verbose = False):
    num_neighbors = len(graph.nodes()) - 1
    for n in graph.nodes():
        neighbors = len(graph.neighbors(n))
        if neighbors != num_neighbors:
            raise AssertionError('The graph is not complete')
    
    source = graph.edges()[0][0]
    destination = graph.edges()[0][1]
    edge = graph.edge[source][destination]
    matches, _ = edge.clean(clean_keys = ['fundamental'])
    kps = edge.source.get_keypoint_coordinates(index=matches['source_idx'], homogeneous=True)
    
    intersection_poly =  (graph, clean_keys)
    intersection_points = intersection_poly.GetGeometryRef(0).GetPoints()
    
    centroid = intersection_poly.Centroid().GetPoint()
    
    points = np.asarray(kps)
    voronoi_np = []
    
    point_cloud = ogr.Geometry(ogr.wkbMultiPoint)
    for p in points:
        point = ogr.Geometry(ogr.wkbPoint)
        point.AddPoint(double(p[0]), double(p[1]))
        point_cloud.AddGeometry(point)
    
    intersection_cloud = intersection_poly.Intersection(point_cloud)
    
    for p in intersection_cloud:
        point = p.GetPoint(0)
        voronoi_np.append(point)
    
    keypoints = np.asarray(voronoi_np)
    voronoi_pd = pd.DataFrame(data = voronoi_np, columns = ['x','y','homogenious'])
    
    # Based on the keypoints found in the method
    inters = np.empty((len(intersection_points),2))
    
    for g, (i, j) in enumerate(intersection_points):
        scaledx, scaledy = scale_poly((i, j), centroid, s)
        point = np.array([scaledx, scaledy])
        inters[g] = point
        
    keypoints = np.vstack((keypoints[:,:2], inters))
    vor = Voronoi(keypoints)
    
    voronoi_df = pd.DataFrame(columns=["x", "y", "weights"])
    voronoi_df["x"] = kps['x']
    voronoi_df["y"] = kps['y']
    
    i = 0
    poly_array = []
    vor_points = np.asarray(vor.points)
    for region in vor.regions:
        region_point = vor_points[np.argwhere(vor.point_region==i)]
        if not -1 in region:
            polygon_points = [vor.vertices[i] for i in region]
            if len(polygon_points) != 0:
                polygon = array_to_poly(polygon_points)
                intersection = polygon.Intersection(intersection_poly)
                poly_array = np.append(poly_array, intersection)  # To see unclipped, replace intersection with polygon
                voronoi_df.loc[(voronoi_df["x"] == region_point[0][0][0]) & 
                               (voronoi_df["y"] == region_point[0][0][1]), 
                               'weights'] = intersection.GetArea()
        i+=1
        
    # Also for visualization of voronoi, adds the colored voronoi polygons to the image
    matplotlib.pyplot.scatter(voronoi_pd['x'], voronoi_pd['y'], color = 'black')
    polygons = poly_array
    for i in polygons:
        geom = i.GetGeometryRef(0)
        if geom == None:
            continue
        else:
            points = geom.GetPoints()
            plt.fill(*zip(*points), alpha = .3)
    # End visualization section
#     plot_vor(graph, edge.source, poly_array, intersection_points, voronoi_pd)

In [11]:
#Point to the adjacency Graph
adjacency = get_path('vor_adjacency.json')
basepath = get_path('Apollo15')
cg = CandidateGraph.from_adjacency(adjacency, basepath=basepath)

#Apply SIFT to extract features
cg.extract_features(method='sift', extractor_parameters={'nfeatures':800})

#Match
cg.match_features()

#Apply outlier detection
# cg.apply_func_to_edges(Edge.symmetry_check)
cg.symmetry_checks()
# cg.apply_func_to_edges(Edge.ratio_check)
cg.ratio_checks()

#Compute a homography and apply RANSAC
cg.apply_func_to_edges("compute_fundamental_matrix", clean_keys=['ratio', 'symmetry'])
# cg.plot()

In [12]:
cliques = list(nx.cliques_containing_node(cg, nodes=1))
print(cliques)
for g in cliques:
    subgraph = cg.create_node_subgraph(g)
    print(cg.edges())
    source = subgraph.edges()[0][0]
    destination = subgraph.edges()[0][1]
    edge = subgraph.edge[source][destination]
    vor(subgraph, clean_keys=['fundamental'])
    matches, mask = edge.clean(clean_keys = ['fundamental'])
    edge.source.plot(index_mask = matches['source_idx'], color = 'black', s = 0, alpha = 0)
    show()

[[1, 0], [1, 4], [1, 5, 6, 7]]
[(0, 1), (2, 3), (2, 4), (3, 4), (4, 1), (5, 1), (5, 6), (5, 7), (6, 1), (6, 7), (1, 7)]


AttributeError: 'tuple' object has no attribute 'GetGeometryRef'

In [ ]:
adjacency = get_path('cube_adjacency.json')
basepath = get_path('Apollo15')
cg = CandidateGraph.from_adjacency(adjacency, basepath=basepath)

#Apply SIFT to extract features
cg.extract_features(method='sift', extractor_parameters={'nfeatures':1000})

#Match
cg.match_features()

#Apply outlier detection
cg.symmetry_checks()
cg.ratio_checks()

#Compute a homography and apply RANSAC
cg.compute_fundamental_matrices(clean_keys=['ratio', 'symmetry'])

cg.compute_homographies(clean_keys = ['fundamental'], reproj_threshold = .5)

edge = cg.edge[0][2]
subgraph = cg.create_node_subgraph([edge.source.node_id, edge.destination.node_id])
vor(subgraph, clean_keys=['fundamental'])
matches, mask = edge.clean(clean_keys = ['fundamental'])
edge.source.plot(index_mask = matches['source_idx'], color = 'black', s = 0, alpha = 0)
show()

In [ ]:
def plot_cluster(graph, ax=None, cmap='Spectral'):
    if ax is None:
        ax = plt.gca()

    if not hasattr(graph, 'clusters'):
        raise AttributeError('Clusters have not been computed.')

    cmap = matplotlib.cm.get_cmap(cmap)

    colors = []

    for i, n in graph.nodes_iter(data=True):
        for j in graph.clusters:
            if i in graph.clusters.get(j):
                print(cmap(j))
                colors.append(cmap(j)[0])
                continue

    nx.draw(graph, ax=ax, node_color=colors)
    return ax